## NHD Network Analysis Demo
___TL;DR___: *We are trying to parallelize hydraulic calculations for dynamic subsets of the U.S. river and stream network*<br><br>
The following was developed as part of the process of preparing a method for forecasting flows on the US network of rivers and streams as represented in the National Hydrography Dataset (NHD). The NHD is a continuously evolving characterization of a fractal system so we felt that we needed to plan to have some flexibility. We hope to identify the complexity inherent in the network at different levels of resolution and we hope to be able to do so dynamically. The goal is also to be able to manage the complexity calculation for arbitrary collections of headwater points, such as might be obtained from a list of named streams or during a major flood event in a particular region.<br>
As a point of terminology, we use the word 'routing' as shorthand to refer to the computation of the translation of a particular flow condition, high or low, to downstream (or in some cases upstream) areas of influence.
The network complexity is related to the potential for parallelization of a serial analysis of the network. We have identified three levels of parallelization that may be implemented: 
1. System-level parallelization of independent networks -- the routing computations for the Mississippi River have little (nothing, except conceptual similarity and a shared existence on earth) to do with the computations for the Columbia river for any practical level of analysis. The system of networks across the US is what we are considering in general.
1. Network-level parallelization of interconnected reaches -- There is a need to consider the computations for adjacent branches within a network of con-flowing streams, but with proper ordering, some of the computations may be considered in parallel. For example, the Illinois River headwaters and the Mississippi River headwaters are related within their broader Mississippi network, but the routing calculations for those headwaters are pratically agnostic to one another.
1. Reach-level parallelization of the specific routing computation -- the numerical work of routing water downstream is a matrix computation and consists of exploring solutions to differential equations, all of which may potentially be examined in parallel, under the proper conditions and with suitable assumptions.<br>


### Import the git repo including test data
This git repo is a fork/branch of the national water model public repository hosted by UCAR. The UCAR repo is the basis for the WRF-Hydro model that is presently the modeling engine of the [US National Water Model.](https://water.noaa.gov/about/nwm)<br>

The network analysis code assumes that the downstream neighbor is identified in the table for each stream segment as is the case for the test datasets. 

We recognize that others have done similar work and may possibly have done it better. We are working on being more able to nimbly respond to suggestions and opportunities for improvement. Please let us know if you see something we could do better or of course feel free to fork and improve what you see.

In [ ]:
import sys
try:
    import google.colab
    ENV_IS_CL = True
    !git clone --single-branch --branch network https://github.com/jameshalgren/wrf_hydro_nwm_public.git
    sys.path.append('/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/src')
    !pip install geopandas
except:
    ENV_IS_CL = False
    sys.path.append(r'../src')


### Create some general functions
The next three blocks define interaction with the `networkbuilder` module in the git repo, which is the tool for creating the `connnections` object to characterize the network. 

In [ ]:
import networkbuilder as networkbuilder
import recursive_print
import os
import geopandas as gpd
# -*- coding: utf-8 -*-
"""NHD Network traversal

A demonstration version of this code is stored in this Colaboratory notebook:
https://colab.research.google.com/github/jameshalgren/wrf_hydro_nwm_public/blob/network/trunk/NDHMS/dynamic_channel_routing/notebooks/NHD_Network_Density_Analysis.ipynb#scrollTo=h_BEdl4LID34

"""
def do_network(
        geofile_path = None
        , title_string = None
        , layer_string = None
        , driver_string = None
        , key_col = None
        , downstream_col = None
        , length_col = None
        , terminal_code = None
        , verbose = False
        , debuglevel = 0
        ):

    # NOTE: these methods can lose the "connections" and "rows" arguments when
    # implemented as class methods where those arrays are members of the class.
    if verbose: print(title_string)
    if debuglevel <= -1: print(f'reading -- dataset: {geofile_path}; layer: {layer_string}; fiona driver: {driver_string}')
    geofile = gpd.read_file(geofile_path, driver=driver_string, layer=layer_string)
    if debuglevel <= -1: print(geofile.head())
    geofile_rows = geofile.to_numpy()
    if debuglevel <= -2: geofile.plot()
    # Kick off recursive call for all connections and keys
    (connections) = networkbuilder.get_down_connections(
                    rows = geofile_rows
                    , key_col = key_col
                    , downstream_col = downstream_col
                    , length_col = length_col
                    , verbose = verbose
                    , debuglevel = debuglevel)
    
    (all_keys, ref_keys, headwater_keys
        , terminal_keys
        , terminal_ref_keys
        , circular_keys) = networkbuilder.determine_keys(
                    connections = connections
                    , rows = geofile_rows
                    , key_col = key_col
                    , downstream_col = downstream_col
                    , terminal_code = terminal_code
                    , verbose = verbose
                    , debuglevel = debuglevel)
    
    (junction_keys) = networkbuilder.get_up_connections(
                    connections = connections
                    , terminal_code = terminal_code
                    , headwater_keys = headwater_keys
                    , terminal_keys = terminal_keys
                    , verbose = verbose
                    , debuglevel = debuglevel)
    return connections, all_keys, ref_keys, headwater_keys \
        , terminal_keys, terminal_ref_keys \
        , circular_keys, junction_keys

In [ ]:
def do_print():    
    recursive_print.print_basic_network_info(
                    connections = connections_NHD
                    , headwater_keys = headwater_keys_NHD
                    , junction_keys = junction_keys_NHD
                    , terminal_keys = terminal_keys_NHD
                    , terminal_code = terminal_code_NHD
                    , verbose = True
                    )
    
    if 1 == 0: #THE RECURSIVE PRINT IS NOT A GOOD IDEA WITH LARGE NETWORKS!!!
        recursive_print.print_connections(
                    headwater_keys = headwater_keys_NHD
                    , down_connections = connections_NHD
                    , up_connections = connections_NHD
                    , terminal_code = terminal_code_NHD
                    , terminal_keys = terminal_keys_NHD
                    , terminal_ref_keys = terminal_ref_keys_NHD
                    , debuglevel = -2
                    )
    


### Build a test case
The `test_rows` object simulates a river network dataset such as we recieve from the National Hydrography Dataset. Each data row has a node ID, a 'to' node ID, and some other relevant data. For this test dataset, the second data column is a dummy length (and the last column could be some other value, but we haven't tried anything yet... stay tuned) and in our traversals, we can add up the lengths as a surrogate for more complex water routing functions we need to eventually manage.

In [ ]:
# def main():
if 1 == 1:
    """##TEST"""
    print("")
    print ('Executing Test')
    # Test data
    test_rows = [
        [50,178,51,0],
        [51,178,50,0],
        [60,178,61,0],
        [61,178,62,0],
        [62,178,60,0],
        [70,178,71,0],
        [71,178,72,0],
        [72,178,73,0],
        [73,178,70,0],
        [80,178,81,0],
        [81,178,82,0],
        [82,178,83,0],
        [83,178,84,0],
        [84,178,80,0],
        [0,456,-999,0],
        [1,178,4,0],
        [2,394,0,0],
        [3,301,2,0],
        [4,798,0,0],
        [5,679,4,0],
        [6,523,0,0],
        [7,815,2,0],
        [8,841,-999,0],
        [9,514,8,0],
        [10,458,9,0],
        [11,832,10,0],
        [12,543,11,0],
        [13,240,12,0],
        [14,548,13,0],
        [15,920,14,0],
        [16,920,15,0],
        [17,514,16,0],
        [18,458,17,0],
        [19,832,18,0],
        [20,543,19,0],
        [21,240,16,0],
        [22,548,21,0],
        [23,920,22,0],
        [24,240,23,0],
        [25,548,12,0],
        [26,920,25,0],
        [27,920,26,0],
        [28,920,27,0],
    ]

    test_key_col = 0
    test_downstream_col = 2
    test_length_col = 1
    test_terminal_code = -999
    debuglevel = 0
    verbose = True

    (test_connections) = networkbuilder.get_down_connections(
                rows = test_rows
                , key_col = test_key_col
                , downstream_col = test_downstream_col
                , length_col = test_length_col
                , verbose = True
                , debuglevel = debuglevel
                )

    (test_all_keys, test_ref_keys, test_headwater_keys
     , test_terminal_keys
     , test_terminal_ref_keys
     , test_circular_keys) = networkbuilder.determine_keys(
                connections = test_connections
                , rows = test_rows
                , key_col = test_key_col
                , downstream_col = test_downstream_col
                , terminal_code = test_terminal_code
                , verbose = True
                , debuglevel = debuglevel
                )

    test_junction_keys = networkbuilder.get_up_connections(
                connections = test_connections
                , terminal_code = test_terminal_code
                , headwater_keys = test_headwater_keys
                , terminal_keys = test_terminal_keys
                , verbose = True
                , debuglevel = debuglevel
                )

    recursive_print.print_connections(
                headwater_keys = test_headwater_keys
                , down_connections = test_connections
                , up_connections = test_connections
                , terminal_code = test_terminal_code
                , terminal_keys = test_terminal_keys
                , terminal_ref_keys = test_terminal_ref_keys
                , debuglevel = debuglevel
                )

    recursive_print.print_basic_network_info(
                connections = test_connections
                , headwater_keys = test_headwater_keys
                , junction_keys = test_junction_keys
                , terminal_keys = test_terminal_keys
                , terminal_code = test_terminal_code
                , verbose = True
                , debuglevel = debuglevel
                )


# if __name__ == "__main__":
#     main()


### Real Networks
(you can skip this cell to test the code on the simple case generated above...)

In [ ]:
if ENV_IS_CL: root = '/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/'
elif not ENV_IS_CL: root = os.path.dirname(os.path.abspath(''))
test_folder = os.path.join(root, r'test')
geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')

"""##NHD Subset (Brazos/Lower Colorado)"""
Brazos_LowerColorado_ge5 = True
"""##NHD CONUS order 5 and greater"""
CONUS_ge5 = True
"""These are large -- be careful"""
CONUS_Named_Streams = False 
CONUS_FULL_RES = False
CONUS_Named_combined = False #process the Named streams through the Full-Res paths to join the many hanging reaches

debuglevel = -1
verbose = True

# The following datasets are extracts from the feature datasets available
# from https://www.nohrsc.noaa.gov/pub/staff/keicher/NWM_live/web/data_tools/
# the CONUS_ge5 and Brazos_LowerColorado_ge5 datasets are included
# in the github test folder. The CONUS_Named_Streams and CONUS_FULL_RES are versions 
# of the data in the nohrsc-hosted archive but are too large to efficiently 
# package inside of the repository. 

if Brazos_LowerColorado_ge5:
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'NHD_BrazosLowerColorado_Channels.shp')
    key_col_NHD = 2
    downstream_col_NHD = 7
    length_col_NHD = 6
    terminal_code_NHD = 0
    title_string = 'Brazos + Lower Colorado\nNHD stream orders 5 and greater\n'
    title_string = 'CONUS Order 5 and Greater '
    driver_string = 'ESRI Shapefile'
    layer_string = 0

    Brazos_LowerColorado_ge5_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

if CONUS_ge5:
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'NHD_Conus_Channels.shp')
    key_col_NHD = 1
    downstream_col_NHD = 6
    length_col_NHD = 5
    terminal_code_NHD = 0
    title_string = 'CONUS Order 5 and Greater '
    driver_string = 'ESRI Shapefile'
    layer_string = 0

    CONUS_ge5_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

if CONUS_Named_Streams:
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'channels_nwm_v12_routeLink_NamedOnly.shp')
    key_col_Named_Streams = 0
    downstream_col_Named_Streams = 5
    length_col_Named_Streams = 4
    terminal_code_Named_Streams = 0
    title_string = 'NHD v1.2 segments corresponding to NHD 2.0 GNIS labeled streams\n'
    # driver_string = 'FileGDB'
    driver_string = 'ESRI Shapefile'
    # layer_string = 'named_streams_v12'
    layer_string = 0

    CONUS_Named_Streams_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_Named_Streams
                , downstream_col = downstream_col_Named_Streams
                , length_col = length_col_Named_Streams
                , terminal_code = terminal_code_Named_Streams
                , verbose = verbose
                , debuglevel = debuglevel)

if CONUS_FULL_RES:
    # nhd_conus_file_path = '../../../../../../GISTemp/nwm_v12.gdb'
    nhd_conus_file_path = os.path.join(geo_input_folder
            , r'channels_nwm_v12_routeLink_all.shp')
    key_col_NHD = 0
    downstream_col_NHD = 5
    length_col_NHD = 4
    terminal_code_NHD = 0
    title_string = 'CONUS Full Resolution NWM v1.2'
    # driver_string = 'FileGDB'
    driver_string = 'ESRI Shapefile'
    # layer_string = 'channels_nwm_v12_routeLink'
    layer_string = 0

    CONUS_FULL_RES_values = do_network (nhd_conus_file_path
                , title_string = title_string
                , layer_string = layer_string
                , driver_string = driver_string
                , key_col = key_col_NHD
                , downstream_col = downstream_col_NHD
                , length_col = length_col_NHD
                , terminal_code = terminal_code_NHD
                , verbose = verbose
                , debuglevel = debuglevel)

In [ ]:
from importlib import reload
reload(networkbuilder)
import pickle

CONUS_Named_combined = True

if CONUS_Named_combined:
    ''' NOW Combine the two CONUS analyses by starting with the Named Headwaters
        but trace the network down the Full Resolution NHD. It should only work
        if the other two datasets have been computed.'''
    if not (CONUS_Named_Streams and CONUS_FULL_RES):
        print('WARNING: If this works, you are using old data...')
    
    # Use only headwater keys that are in the full dataset.
    headwater_keys_combined = CONUS_FULL_RES_values[3] & \
                                CONUS_Named_Streams_values[3]
    # Need to make sure that these objects are independent -- we will modify them a bit.
    connections_combined = pickle.loads(pickle.dumps(CONUS_FULL_RES_values[0]))
    terminal_keys_combined = pickle.loads(pickle.dumps(CONUS_FULL_RES_values[4]))
    terminal_code_combined = terminal_code_NHD
    
    for key in connections_combined: #Clear the upstreams and rebuild it with just named streams
        connections_combined[key].pop('upstreams',None)
        

In [ ]:

    (junction_keys_combined
     , visited_keys_combined
     , visited_terminal_keys_combined
     , junction_count_combined) = networkbuilder.get_up_connections(
                 connections = connections_combined
                 , terminal_code = terminal_code_combined
                 , headwater_keys = headwater_keys_combined
                 , terminal_keys = terminal_keys_combined
                 , verbose = verbose
                 , debuglevel = debuglevel)
    
#     print(len(junction_keys_combined)
#      , len(visited_keys_combined)
#      , len(visited_terminal_keys_combined)
#      , junction_count_combined)
    
#     print(len(terminal_keys_combined - visited_terminal_keys_combined))

### With this, we can separate the different rivers in the network
Once a 'connection' object has been created with a representation of the river network, we can traverse that object and perform calculations -- in the example below, we parallelize the process of traversing the independent portions of the network and then serially compute the number of junctions. This corresponds to the "**System-level parallelization**" mentioned as _item 1_ above.
### NOW for the next step
We could compute total upstream length or (and this is the real goal) flow due to incoming lateral contributions from the land accumulated over the entire upstream network. That second calculation can also be parallelized but we have to figure out how to accomplish intelligently so that the collective calculation is network-aware. Such a parallelization would be the "**Network-level parallelization of interconnected reaches**" mentioned as _item 2_ above. The upstream length will depend on the number of upstream branches and their configuration, so there has to be some concept of stream order and topology built into the parallelization method.

In [ ]:

#parallel compute
import time
import multiprocessing
from functools import partial

'''Test'''
# terminal_code = test_terminal_code
# terminal_keys = test_terminal_keys 
# circular_keys = test_circular_keys
# terminal_keys_super = terminal_keys - circular_keys
# con = test_connections
'''Streams of NHD order 5 or greater confluent to the Brazos and Lower Colorado'''
# terminal_code = terminal_code_NHD
# terminal_keys = Brazos_LowerColorado_ge5_values[4] 
# circular_keys = Brazos_LowerColorado_ge5_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = Brazos_LowerColorado_ge5_values[0]
'''CONUS streams of NHD order 5 or greater'''
terminal_code = terminal_code_NHD
terminal_keys = CONUS_ge5_values[4] 
circular_keys = CONUS_ge5_values[6]
terminal_keys_super = terminal_keys - circular_keys
con = CONUS_ge5_values[0]
'''Named Streams'''
# terminal_keys = CONUS_Named_Streams_values[4] 
# circular_keys = CONUS_Named_Streams_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_Named_Streams_values[0]
# terminal_code = terminal_code_NHD
'''Full Res NHD'''
# terminal_keys = CONUS_FULL_RES_values[4] 
# circular_keys = CONUS_FULL_RES_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = CONUS_FULL_RES_values[0]
# terminal_code = terminal_code_NHD
'''Combined'''
# # # NOT NEEDED # terminal_keys = terminal_keys_combined 
# # # NOT NEEDED # circular_keys = circular_keys_combined
# terminal_keys_super = visited_terminal_keys_combined
# con = connections_combined
# terminal_code = terminal_code_NHD

def recursive_junction_read (
                             keys
                             , network
                             , terminal_code = 0
                             , verbose = False
                             , debuglevel = 0
                            ):
    global con
    for key in keys:
        ckey = key
        try:
            ukeys = con[key]['upstreams']
            while not len(ukeys) >= 2 and not (ukeys == {terminal_code}):
                if debuglevel <= -3: print(f"segs at ckey {ckey}: {network['segment_count']}")
                # the terminal code will indicate a headwater
                if debuglevel <= -3: print(ukeys)
                (ckey,) = ukeys
                ukeys = con[ckey]['upstreams']
            if ukeys == {terminal_code}:
                if debuglevel <= -2: print(f"headwater found at {ckey}")
                network['segment_count'] += 1
                if debuglevel <= -3: print(f"segs at ckey {ckey}: {network['segment_count']}")
            elif len(ukeys) >= 2:
                network['segment_count'] += 1
                if debuglevel <= -2: print(f"junction found at {ckey} with upstreams {ukeys}")
                network['segment_count'] += 1
                if debuglevel <= -3: print(f"segs at ckey {ckey}: {network['segment_count']}")
                network['junction_count'] += 1 #the Terminal Segment
                recursive_junction_read (ukeys, network, terminal_code = terminal_code, verbose = verbose, debuglevel = debuglevel) 
                # print(ukeys)
                ukeys = con[ckey]['upstreams']
                ckey = ukeys
        except:
            if debuglevel <= -2: 
                print(f'There is a problem with connection: {key}: {con[key]}')

def super_network_trace(
                        nid
                        , terminal_code = terminal_code
                        , verbose= False
                        , debuglevel = 0
                        ):

    network = {}
    global con
    us_length_total = 0
    
    if verbose: print(f'\ntraversing upstream on network {nid}:')
    # try:
    if 1 == 1:
        network.update({'junction_count': 0})
        network.update({'segment_count': 0}) #the Terminal Segment
        recursive_junction_read([nid], network, verbose = verbose, terminal_code = terminal_code, debuglevel = debuglevel)
        if verbose: print(f"junctions: {network['junction_count']}")
        if verbose: print(f"segments: {network['segment_count']}")
    # except Exception as exc:
    #     print(exc)
    return {nid: network, 'upstream_length': us_length_total}



In [ ]:
###continuing from previous cell
super_networks = {terminal_key:{}
                        for terminal_key in terminal_keys_super}
debuglevel = -1
verbose = False

start_time = time.time()
results_serial = {}
for nid, network in super_networks.items():
    network.update(super_network_trace(nid, terminal_code = terminal_code, verbose = verbose, debuglevel = debuglevel)[nid])
print("--- %s seconds: serial compute ---" % (time.time() - start_time))
if debuglevel <= -1: print(len(super_networks.items()))
if debuglevel <= -2: print(super_networks)

    ## Notice that I'm not timing the initialization in each case, 
    ## which might be considered cheating a little bit. 
    ## I timed it for the first case for reference.
nids = (nid for nid in super_networks)
start_time = time.time()
with multiprocessing.Pool() as pool:
    print("--- %s seconds: parallel overhead to load multiprocessing.Pool() ---" % (time.time() - start_time))    
    start_time = time.time()
    results = pool.map(super_network_trace, nids)
    print("--- %s seconds: parallel compute using default terminal_code ---" % (time.time() - start_time))
if debuglevel <= -1: print(len(results))
if debuglevel <= -2: print(results)

nids = (nid for nid in super_networks)
snt = partial(super_network_trace, terminal_code = terminal_code)
with multiprocessing.Pool() as pool:
    start_time = time.time()
    results = pool.map(snt, nids)
    print("--- %s seconds: parallel compute using partial function ---" % (time.time() - start_time))
if debuglevel <= -1: print(len(results))
if debuglevel <= -2: print(results)

nidsWtc = ([nid,terminal_code] for nid in super_networks)
with multiprocessing.Pool() as pool:
    start_time = time.time()
    results = pool.starmap(super_network_trace, nidsWtc)
    print("--- %s seconds: parallel compute with list of lists and starmap ---" % (time.time() - start_time))
if debuglevel <= -1: print(len(results))
if debuglevel <= -2: print(results)
